Como experto en diseño de basas de datos por favor diseña un esquema de base de datos para una aplicación de comercio electrónico con tablas para usuarios, productos, pedidos y order_item

ChatGPT dijo:

Claro, aquí tienes un diseño de base de datos relacional para una aplicación de comercio electrónico básica. El diseño incluye las siguientes entidades: usuarios, productos, pedidos y order_item (ítems de pedido). A continuación te muestro el esquema con claves primarias, foráneas y algunos atributos relevantes.

## 🧩 Esquema de la base de datos

- 1. usuarios

Información de los clientes registrados.

```sql
CREATE TABLE usuarios (
    id_usuario        INT PRIMARY KEY AUTO_INCREMENT,
    nombre            VARCHAR(100) NOT NULL,
    email             VARCHAR(100) UNIQUE NOT NULL,
    contraseña_hash   VARCHAR(255) NOT NULL,
    direccion         TEXT,
    telefono          VARCHAR(20),
    fecha_creacion    TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

- 2. productos

Catálogo de productos disponibles para la venta.

```sql
CREATE TABLE productos (
    id_producto       INT PRIMARY KEY AUTO_INCREMENT,
    nombre            VARCHAR(100) NOT NULL,
    descripcion       TEXT,
    precio            DECIMAL(10,2) NOT NULL,
    stock             INT NOT NULL,
    imagen_url        VARCHAR(255),
    activo            BOOLEAN DEFAULT TRUE,
    fecha_creacion    TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

- 3. pedidos

Representa cada compra realizada por un usuario.

```sql
CREATE TABLE pedidos (
    id_pedido         INT PRIMARY KEY AUTO_INCREMENT,
    id_usuario        INT NOT NULL,
    fecha_pedido      TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    estado            ENUM('pendiente', 'pagado', 'enviado', 'cancelado', 'entregado') DEFAULT 'pendiente',
    total             DECIMAL(10,2) NOT NULL,
    
    FOREIGN KEY (id_usuario) REFERENCES usuarios(id_usuario)
        ON DELETE CASCADE
);
```

- 4. order_item

Asocia productos a un pedido, con cantidad y subtotal.

```sql
CREATE TABLE order_item (
    id_item           INT PRIMARY KEY AUTO_INCREMENT,
    id_pedido         INT NOT NULL,
    id_producto       INT NOT NULL,
    cantidad          INT NOT NULL,
    precio_unitario   DECIMAL(10,2) NOT NULL,  -- precio al momento de la compra
    subtotal          DECIMAL(10,2) NOT NULL,  -- cantidad * precio_unitario

    FOREIGN KEY (id_pedido) REFERENCES pedidos(id_pedido)
        ON DELETE CASCADE,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
        ON DELETE CASCADE
);
```

- 🔁 Relaciones

  - usuarios - pedidos: 1 a N → un usuario puede tener muchos pedidos.
  - pedidos - order_item: 1 a N → un pedido tiene múltiples productos.
  - productos - order_item: N a 1 → un producto puede estar en muchos pedidos.

- ✅ Consideraciones adicionales

  - Puedes agregar una tabla pagos si deseas manejar distintos métodos o estados de pago.
  - Para un sistema multivendedor, agregar una tabla vendedores y asociar productos a vendedores.
  - Si manejas categorías de productos, puedes crear una tabla categorias y una relación producto_categoria.
  - Si deseas auditar cambios, considera timestamps y campos como modificado_por.

¿Te gustaría que convierta esto en un modelo visual (ERD)? ¿O necesitas la versión en ORM como SQLAlchemy o Django models?

Dijiste:
podes darme el diseño como un uml

ChatGPT dijo:
¡Claro! Aquí tienes el diseño de la base de datos como un diagrama UML de clases, que representa la estructura y relaciones entre Usuario, Producto, Pedido y OrderItem.

- 📘 Diagrama UML (versión textual)

```sql
+---------------------+
|     Usuario         |
+---------------------+
| - id_usuario: int   |
| - nombre: string    |
| - email: string     |
| - contraseña: string|
| - direccion: string |
| - telefono: string  |
| - fecha_creacion:   |
|    datetime         |
+---------------------+
| +crearPedido()      |
+---------------------+
          |
          | 1
          |
          |-----------------------+
                                  |
                                  | *
                          +--------------------+
                          |      Pedido        |
                          +--------------------+
                          | - id_pedido: int   |
                          | - fecha_pedido:    |
                          |     datetime       |
                          | - estado: enum     |
                          | - total: decimal   |
                          | - id_usuario: int  |
                          +--------------------+
                          | +agregarItem()     |
                          +--------------------+
                                  |
                     +------------+-------------+
                     |                          |
                     | *                        | *
        +---------------------+        +---------------------+
        |     OrderItem       |        |     Producto        |
        +---------------------+        +---------------------+
        | - id_item: int      |        | - id_producto: int  |
        | - cantidad: int     |        | - nombre: string    |
        | - precio_unitario:  |        | - descripcion: text |
        |    decimel          |        | - precio: decimal   |
        | - subtotal: decimal |        | - stock: int        |
        | - id_pedido: int    |        | - imagen_url: string|
        | - id_producto: int  |        | - activo: boolean   |
        +---------------------+        | - fecha_creacion: e |
                                       |    datetime         |
                                       +---------------------+
```

- 📌 Relaciones clave:
  - Usuario 1⟶ Pedido*: un usuario puede tener muchos pedidos.
  - Pedido 1⟶ OrderItem*: un pedido puede contener muchos ítems.
  - Producto 1⟶ OrderItem*: un producto puede estar en muchos ítems de pedidos.

OrderItem conecta un producto con un pedido con información adicional como cantidad y precio.

¿Querés el UML como imagen?
Puedo generarlo visualmente o darte el código en lenguaje UML (como PlantUML) para que lo visualices tú mismo en una herramienta como PlantUML Live.

¿Querés el código PlantUML también?

Dijiste:
voy a usar sqlite3 podes adapme el codigo

ChatGPT dijo:
Perfecto. A continuación te adapto el esquema de la base de datos para SQLite3, teniendo en cuenta sus particularidades:

Usa INTEGER PRIMARY KEY AUTOINCREMENT en lugar de AUTO_INCREMENT.

SQLite no tiene un tipo ENUM, así que usamos TEXT con restricción CHECK(...).

No tiene un tipo BOOLEAN nativo, se usa INTEGER con 0 o 1.

DATETIME DEFAULT CURRENT_TIMESTAMP funciona como TIMESTAMP.

- 🛠️ Esquema en SQLite3

```sql
-- Tabla: usuarios
CREATE TABLE usuarios (
    id_usuario        INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre            TEXT NOT NULL,
    email             TEXT NOT NULL UNIQUE,
    contraseña_hash   TEXT NOT NULL,
    direccion         TEXT,
    telefono          TEXT,
    fecha_creacion    DATETIME DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: productos
CREATE TABLE productos (
    id_producto       INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre            TEXT NOT NULL,
    descripcion       TEXT,
    precio            REAL NOT NULL,
    stock             INTEGER NOT NULL,
    imagen_url        TEXT,
    activo            INTEGER DEFAULT 1,  -- 1 = TRUE, 0 = FALSE
    fecha_creacion    DATETIME DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: pedidos
CREATE TABLE pedidos (
    id_pedido         INTEGER PRIMARY KEY AUTOINCREMENT,
    id_usuario        INTEGER NOT NULL,
    fecha_pedido      DATETIME DEFAULT CURRENT_TIMESTAMP,
    estado            TEXT CHECK(estado IN ('pendiente', 'pagado', 'enviado', 'cancelado', 'entregado')) DEFAULT 'pendiente',
    total             REAL NOT NULL,
    FOREIGN KEY (id_usuario) REFERENCES usuarios(id_usuario) ON DELETE CASCADE
);

-- Tabla: order_item
CREATE TABLE order_item (
    id_item           INTEGER PRIMARY KEY AUTOINCREMENT,
    id_pedido         INTEGER NOT NULL,
    id_producto       INTEGER NOT NULL,
    cantidad          INTEGER NOT NULL,
    precio_unitario   REAL NOT NULL,
    subtotal          REAL NOT NULL,
    FOREIGN KEY (id_pedido) REFERENCES pedidos(id_pedido) ON DELETE CASCADE,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE CASCADE
);
```

- ✅ Recomendaciones para SQLite3

Usa 

```sql
PRAGMA foreign_keys = ON; 
```

Cada vez que abras conexión si querés que se respeten las claves foráneas (por defecto están desactivadas).
Si necesitás índices adicionales para mejorar el rendimiento, se pueden crear con `CREATE INDEX`.

¿Querés que te genere también un script Python para crear estas tablas usando sqlite3?